In [ ]:
import praw
import json
import time
import os

# Initialize Reddit instance with credentials
reddit = praw.Reddit(
    client_id="R4aTlqzdkwL8HlP0kbqI_w",            # Replace with your client_id
    client_secret="_-xw-M0CWgf3xDc7XMJU3RdCWB9WIQ", # Replace with your client_secret
    user_agent="Hazel/1.0 by SeaLimit6194"
)

# New list of non-overlapping subreddits covering unique topics
subreddits = [
    "woodworking", "astrophotography", "birding", "urbanplanning", "archaeology", 
    "mycology", "linguistics", "boardgames", "tabletopgamedesign", "lego", 
    "bicycling", "rockclimbing", "astrology", "pagan", "tarot", 
    "aviation", "nasa", "paleontology", "conlangs", "speculativeevolution", 
    "beekeeping", "bonsai", "canning", "crafts", "embroidery", 
    "metalworking", "lockpicking", "trains", "campingandhiking", "golf", 
    "knitting", "mechanicalkeyboards", "vintagecomputing", "classiccars", "vintagemotorcycles", 
    "overlanding", "climbing", "ultramarathon", "homebrewing", "whiskey", 
    "sailing", "surfing", "diving", "freediving", "underwaterphotography", 
    "sneakers", "streetwear", "rawdenim", "fountainpens", "minimalism", 
    "budgettravel", "solotravel", "digitalnomad", "productivity", "minimalism", 
    "cookingforbeginners", "BBQ", "sousvide", "baking", "tea", 
    "homelab", "networking", "sysadmin", "netsec", "cloudcomputing", 
    "datacenter", "kubernetes", "devops", "pythonforengineers", "datascience", 
    "geology", "mining", "oilandgasworkers", "renewableenergy", "mechanicalengineering", 
    "electricalengineering", "chemicalengineering", "composting", "hydroponics", "urbanfarming", 
    "mushroomgrowers", "vegan", "plantbased", "herbalism", "nutritionfacts", 
    "addiction", "adhd", "anxiety", "mentalhealth", "depression"
]

# Categories to retrieve posts from
categories = ["hot", "new", "rising", "controversial", "top"]

# Settings
posts_per_subreddit = 500
output_file = "redditdata1.json"
save_interval = 500  # Save data every 500 posts to reach the target size faster

# Load existing data and IDs to avoid duplicates
if os.path.exists(output_file):
    with open(output_file, "r") as f:
        data = json.load(f)
    existing_ids = {post['id'] for post in data if 'id' in post}
else:
    data = []
    existing_ids = set()

# Fetch function with deduplication
def fetch_subreddit_posts(subreddit_name, category):
    subreddit = reddit.subreddit(subreddit_name)
    posts = []
    
    try:
        submissions = getattr(subreddit, category)(limit=posts_per_subreddit)
        
        for submission in submissions:
            if submission.id not in existing_ids:
                post_data = {
                    "id": submission.id,
                    "title": submission.title,
                    "score": submission.score,
                    "author": str(submission.author),
                    "created_utc": submission.created_utc,
                    "subreddit": subreddit.display_name,
                    "selftext": submission.selftext,
                    "url": submission.url,
                    "category": category
                }
                posts.append(post_data)
                existing_ids.add(submission.id)
    except Exception as e:
        print(f"Error fetching {category} posts from r/{subreddit_name}: {e}")
    return posts

# Data collection with periodic saving and size limit
post_count = 0
for _ in range(50):  # Increased loop count to accumulate more data
    for subreddit_name in subreddits:
        for category in categories:
            print(f"Fetching {category} posts from r/{subreddit_name}...")
            new_posts = fetch_subreddit_posts(subreddit_name, category)
            data.extend(new_posts)
            post_count += len(new_posts)
            time.sleep(2)  # Slightly shorter delay to increase data fetching

            # Save periodically to avoid data loss
            if post_count >= save_interval:
                with open(output_file, "w") as f:
                    json.dump(data, f, indent=4)
                    f.flush()
                print(f"Saved {post_count} posts to {output_file}")
                post_count = 0  # Reset counter after save

            # Check file size limit
            if os.path.getsize(output_file) >= 500 * 1024 * 1024:  # 500 MB
                print("Target file size of 500 MB reached. Stopping...")
                break
        else:
            continue
        break

print(f"Data collection complete. Saved to {output_file}")


Fetching hot posts from r/woodworking...
Fetching new posts from r/woodworking...
Fetching rising posts from r/woodworking...
Fetching controversial posts from r/woodworking...
Fetching top posts from r/woodworking...
Fetching hot posts from r/astrophotography...
Fetching new posts from r/astrophotography...
Fetching rising posts from r/astrophotography...
Fetching controversial posts from r/astrophotography...
Fetching top posts from r/astrophotography...
Fetching hot posts from r/birding...
Fetching new posts from r/birding...
Fetching rising posts from r/birding...
Fetching controversial posts from r/birding...
Fetching top posts from r/birding...
Fetching hot posts from r/urbanplanning...
Fetching new posts from r/urbanplanning...
Fetching rising posts from r/urbanplanning...
Fetching controversial posts from r/urbanplanning...
Fetching top posts from r/urbanplanning...
Fetching hot posts from r/archaeology...
Fetching new posts from r/archaeology...
Fetching rising posts from r/ar